In [28]:
import pandas as pd
import numpy as np
col_names=['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship',
           'race','sex','capital_gain','capital_loss','hours_per_week','native_country','class_']

train = pd.read_csv("adult.data.txt",sep=',',header=None, names=col_names)
test = pd.read_csv("adult.test.txt",sep=',',skiprows=1,header=None,names=col_names)

train['workclass'] = train.workclass.astype('category')
train['education'] = train.education.astype('category')
train['marital_status'] = train.marital_status.astype('category')
train['occupation'] = train.occupation.astype('category')
train['relationship'] = train.relationship.astype('category')
train['race'] = train.race.astype('category')
train['sex'] = train.sex.astype('category')
train['native_country'] = train.native_country.astype('category')
train['class_'] = train.class_.astype('category')

train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'class_'],
      dtype='object')

In [29]:
#Create dummy variables for categorical variables
train_dum = pd.get_dummies(train, prefix='C_', columns=['workclass'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['education'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['marital_status'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['occupation'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['relationship'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['race'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['sex'])
train_dum = pd.get_dummies(train_dum, prefix='C_', columns=['native_country'])

Y_train=train.loc[:,'class_']
Y_train=Y_train.replace(to_replace=' >50K',value=1)
Y_train=Y_train.replace(to_replace=' <=50K',value=0)

X_train=train_dum
X_train.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,class_,C__ ?,C__ Federal-gov,C__ Local-gov,...,C__ Portugal,C__ Puerto-Rico,C__ Scotland,C__ South,C__ Taiwan,C__ Thailand,C__ Trinadad&Tobago,C__ United-States,C__ Vietnam,C__ Yugoslavia
0,39,77516,13,2174,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
#X_train = X_train.drop(columns = ['class_'])
X = X_train
y = Y_train

In [91]:
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [92]:
y_pred = clf.predict(X)
accuracy = sum(y_pred == y)/y.shape[0]
print("Accuracy: {}".format(accuracy))

Accuracy: 0.9999385768250361


In [93]:
test_dum = pd.get_dummies(test, prefix='C_', columns=['workclass'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['education'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['marital_status'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['occupation'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['relationship'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['race'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['sex'])
test_dum = pd.get_dummies(test_dum, prefix='C_', columns=['native_country'])

In [94]:
Y_test=test.loc[:,'class_']
Y_test=Y_test.replace(to_replace=' >50K.',value=1)
Y_test=Y_test.replace(to_replace=' <=50K.',value=0)
X_test = test_dum
#As there is no observation that has a C__ Holand-Netherlands value, the test set dataframe doesn't have this column,
#Therefore, I should add this column back to the test set to make sure the format is the same as training set
idx=X_train.columns.get_loc("C__ Holand-Netherlands")
X_test.insert(loc=idx, column="C__ Holand-Netherlands", value=0)

In [95]:
X_test = X_test.drop(columns = ['class_'])

In [96]:
X_test = X_test
y_test = Y_test
pred_test = clf.predict(X_test)
Accuracy_Test = sum(pred_test == y_test)/y_test.shape[0]
print("Accuracy for the test set: {}".format(Accuracy_Test))

Accuracy for the test set: 0.8520361157177078
